# EDA

In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import dotenv_values
import pandas as pd
config = dotenv_values(".env")
api_key = config["KEY"]

In [5]:
from scrape.tft import TFTScraper

summoner_name = "Oturam"
watcher = TFTScraper(api_key=api_key)

match_history = watcher.get_match_history_summoner_name(summoner_name, count=2)
len(match_history)

2

In [17]:
from etl.extract import data_from_summoner_name
data_from_summoner_name(summoner_name, 2)

,match_id,data_version,level,placement,puuid,Arcanist,Assassin,Bodyguard,Bruiser,Challenger,Colossus,Enchanter,Innovator,Protector,Scholar,Sniper,Transformer,Twinshot,unit_0,unit_1,unit_2,unit_3,unit_4,unit_5,unit_6,unit_7,unit_8,tier_unit_0,tier_unit_1,tier_unit_2,tier_unit_3,tier_unit_4,tier_unit_5,tier_unit_6,tier_unit_7,tier_unit_8
0,BR1_2445087948,5,8,8,CYWXGzvpi2ggHY6AA7mq_zGwdtsvtKKlkU5PsEUC3reyNB...,4,NaN,NaN,4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zac,Malzahar,ChoGath,DrMundo,Kaisa,Viktor,TahmKench,NaN,NaN,1,2,2,2,1,1,1,NaN,NaN
1,BR1_2445087948,5,8,7,9sU_lfxN7AvK8tyRKdcKzG5gWzcHz9LHRXbOaTKJZneX-d...,NaN,3,4,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,Darius,Katarina,Zyra,Shaco,Ekko,Braum,Yuumi,Akali,NaN,2,1,2,1,2,2,1,1,NaN
2,BR1_2445087948,5,8,4,RNAOAfUbuqU9vMHgTkdvuom3IGDSPk8BR1H0jQzX8i4aFx...,NaN,NaN,4,NaN,NaN,NaN,NaN,3,4,NaN,3,1,3,Graves,Caitlyn,Kassadin,Garen,Blitzcrank,KogMaw,Jhin,Jayce,NaN,3,3,3,3,3,2,1,1,NaN
3,BR1_2445087948,5,8,3,raFbU5PamnC1UkFnrzAfdd8OGI9LfK3ERzxAhRdv_EmtgB...,4,NaN,4,NaN,NaN,NaN,4,3,4,3,NaN,NaN,NaN,Poppy,Blitzcrank,Lissandra,Vex,Heimerdinger,Janna,Yuumi,Viktor,NaN,3,2,3,2,3,2,1,2,NaN
4,BR1_2445087948,5,8,6,YcQnR0ZG4xHnXP8B9LnwIwH1QYjxiNvrbm3x4b2Os89gzx...,4,NaN,4,4,4,1,NaN,NaN,4,NaN,3,NaN,3,Kassadin,Blitzcrank,KogMaw,Malzahar,ChoGath,DrMundo,Kaisa,NaN,NaN,2,2,2,3,3,2,1,NaN,NaN
5,BR1_2445087948,5,9,5,5sUMY--M2V_BQAv6dr4kewUTL1OeFUc5BR4NTjpp4r6uCG...,NaN,NaN,NaN,4,4,NaN,NaN,3,NaN,NaN,3,1,3,Illaoi,Caitlyn,Vi,Quinn,MissFortune,Gangplank,Fiora,TahmKench,Jayce,1,1,2,1,2,2,3,2,2
6,BR1_2445087948,5,8,2,AUx12_XDc-eDMs97vZkI8M8y7_WtrLnxL_86yHe8GDKekq...,4,NaN,4,NaN,NaN,NaN,4,3,NaN,3,3,NaN,NaN,Poppy,Ziggs,Tristana,Lulu,Vex,Heimerdinger,Taric,Veigar,NaN,3,3,3,3,3,3,2,2,NaN
7,BR1_2445087948,5,8,1,nic_ehkVUE5owwwsPlwEgXmNej_G8GqIyDqpmzME7-uzlM...,4,3,4,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,Darius,TwistedFate,Zyra,Talon,Shaco,Ekko,Braum,Akali,NaN,2,2,2,2,3,2,2,2,NaN
8,BR1_2445024448,5,8,4,s1QIG8I4nN3Vivf1wlJoeV4Mq4Q4Amj9QayuVTXj1F8RTs...,NaN,NaN,4,NaN,NaN,1,4,NaN,4,3,NaN,NaN,3,Graves,Kassadin,Garen,Blitzcrank,Gangplank,Sion,Janna,NaN,NaN,2,2,2,2,3,2,2,NaN,NaN
9,BR1_2445024448,5,8,3,qZBaCsRkQdAwMuiyokfEOzvhRqtSNvkBAJimM8Zryt_1mI...,NaN,3,4,4,NaN,NaN,4,3,4,3,NaN,1,3,Trundle,Vi,Blitzcrank,Zac,Ekko,Janna,Jinx,Jayce,NaN,3,3,2,2,3,1,1,1,NaN
